#### 时间序列(time series)数据是一种重要的结构化数据形式，应用于多个领 域，包括金融学、经济学、生态学、神经科学、物理学等。在多个时间点观察 或测量到的任何事物都可以形成一段时间序列。很多时间序列是固定频率的， 也就是说，数据点是根据某种规律定期出现的(比如每 15 秒、每 5 分钟、每月 出现一次)。时间序列也可以是不定期的，没有固定的时间单位或单位之间的 偏移量。时间序列数据的意义取决于具体的应用场景，主要有以下几种:


#### 时间戳(timestamp)，特定的时刻。

#### 固定时期(period)，如 2007 年 1 月或 2010 年全年。

#### 时间间隔(interval)，由起始和结束时间戳表示。时期(period)可以被看做间隔(interval)的特例。

#### 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。例如，从放入烤箱时起，每秒钟饼干的直径。

#### 本章主要讲解前 3 种时间序列。许多技术都可用于处理实验型时间序列，其索 引可能是一个整数或浮点数(表示从实验开始算起已经过去的时间)。最简单 也最常见的时间序列都是用时间戳进行索引的。


#### 提示:pandas 也支持基于 timedeltas 的指数，它可以有效代表实验或经过的 时间。这本书不涉及 timedelta 指数，但你可以学习 pandas 的文档 

#### pandas 提供了许多内置的时间序列处理工具和数据算法。因此，你可以高效处 理非常大的时间序列，轻松地进行切片/切块、聚合、对定期/不定期的时间序 列进行重采样等。有些工具特别适合金融和经济应用，你当然也可以用它们来 分析服务器日志数据。


### 11.1 日期和时间数据类型及工具

#### Python 标准库包含用于日期(date)和时间(time)数据的数据类型，而且还 有日历方面的功能。我们主要会用到 datetime、time 以及 calendar 模块。 datetime.datetime(也可以简写为 datetime)是用得最多的数据类型:


In [2]:
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2019, 7, 16, 20, 47, 44, 853033)

In [3]:
now.year,now.month,now.day

(2019, 7, 16)

#### datetime以毫秒形式储存日期和时间。timedelta表示两个datetime对象的时间差：

In [4]:
delta = datetime(2011,1,7)-datetime(2008,6,24,8,15)
delta

datetime.timedelta(days=926, seconds=56700)

In [5]:
delta.days

926

In [10]:
delta.seconds

56700

#### 可以给 datetime 对象加上(或减去)一个或多个 timedelta，这样会产生一个 新对象:

In [6]:
from datetime import timedelta
start = datetime(2011,1,7)
start + timedelta(12)


datetime.datetime(2011, 1, 19, 0, 0)

In [7]:
start - 2*timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

#### datetime 模块中的数据类型参见表 10-1。虽然本章主要讲的是 pandas 数据类 型和高级时间序列处理，但你肯定会在 Python 的其他地方遇到有关 datetime 的数据类型。


#### datetime 模块中的数据类型

#### date       以公历形式存储日历日期（年、月、日）
#### time       将时间存储为时、分、秒、毫秒
#### datetime    存储日期和时间
#### timedate    表示两个datetime值之间的差值

### 字符串和 datetime 的相互转换

#### 利用 str 或 strftime 方法(传入一个格式化字符串)，datetime 对象和 pandas 的 Timestamp 对象(稍后就会介绍)可以被格式化为字符串:


In [8]:
stamp = datetime(2011,1,3)
str(stamp)

'2011-01-03 00:00:00'

In [9]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

#### datetime 格式定义

#### %Y     4位数的年
#### %y     2位数的年
#### %m    2位数的月【01，12】
#### %d     2位数的天【01，31】
#### %H      24小时制【00，23】
####  %I     12小时制【01，12】
####  %M    2位数的分【00，59】
####  %S     秒【00，61】（秒60和秒61用于闰秒）
####  %w    用整数表示的星期几【0，6】
####   %U   每年的第几周【00，53】，星期天被认为是每周的第一天，每年的第一个星期天之前的那几天被认为是第0周
####  %W   每年的第几周【00，53】，星期一被认为是每周的第一天，每年的第一个星期一之前的那几天被认为是第0周
####  %D     %m/%d/%y简写形式   例如04/08/12
####  datetime.strptime 可以用这些格式化编码将字符串转换为日期:

In [10]:
value = '2011-01-03'
datetime.strptime(value,'%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [11]:
datestrs = ['7/6/2011','8/6/2011']
[datetime.strptime(x,'%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

#### datetime.strptime 是通过已知格式进行日期解析的最佳方式。但是每次都要 编写格式定义是很麻烦的事情，尤其是对于一些常见的日期格式。这种情况 下，你可以用 dateutil 这个第三方包中的 parser.parse 方法(pandas 中已经 自动安装好了):


In [12]:
from dateutil.parser import parse 
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

#### dateutil 可以解析几乎所有人类能够理解的日期表示形式:

In [13]:
parse('Jan 31,1997 10:45 PM')


datetime.datetime(2019, 1, 31, 22, 45)

#### 在国际通用的格式中，日出现在月的前面很普遍，传入 dayfirst=True 即可解 决这个问题:


In [14]:
parse('6/12/2011',dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

#### pandas 通常是用于处理成组日期的，不管这些日期是 DataFrame 的轴索引还是 列。to_datetime 方法可以解析多种不同的日期表示形式。对标准日期格式 (如 ISO8601)的解析非常快:



In [15]:
import pandas as pd
datestrs = ['2011-07-06  12:00:00','2011-08-06 00:00:00']
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

#### 它还可以处理缺失值(None、空字符串等):

In [16]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [17]:
idx[2]

NaT

In [18]:
pd.isnull(idx)

array([False, False,  True])

#### 注意:dateutil.parser 是一个实用但不完美的工具。比如说，它会把一些原 本不是日期的字符串认作是日期(比如"42"会被解析为 2042 年的今天)。datetime 对象还有一些特定于当前环境(位于不同国家或使用不同语言的系 统)的格式化选项。例如，德语或法语系统所用的月份简写就与英语系统所用 的不同。表 11-3 进行了总结。


#### 11.2 时间序列基础

#### pandas 最基本的时间序列类型就是以时间戳(通常以 Python 字符串或 datatime 对象表示)为索引的 Series:


In [19]:
import numpy as np
from datetime import datetime
dates = [datetime(2011,1,2),datetime(2011,1,5),datetime(2011,1,7),datetime(2011,1,8),datetime(2011,1,10),datetime(2011,1,12)]
ts = pd.Series(np.random.randn(6),index = dates)
ts

2011-01-02    1.674547
2011-01-05   -0.793054
2011-01-07    0.373030
2011-01-08   -1.286663
2011-01-10   -0.336689
2011-01-12    0.109767
dtype: float64

#### 这些 datetime 对象实际上是被放在一个 DatetimeIndex 中的:

In [20]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

#### 跟其他 Series 一样，不同索引的时间序列之间的算术运算会自动按日期对齐:

In [21]:
ts + ts[::2]

2011-01-02    3.349093
2011-01-05         NaN
2011-01-07    0.746060
2011-01-08         NaN
2011-01-10   -0.673377
2011-01-12         NaN
dtype: float64

#### ts[::2] 是每隔两个取一个。

#### pandas 用 NumPy 的 datetime64 数据类型以纳秒形式存储时间戳:

In [22]:
ts.index.dtype

dtype('<M8[ns]')

#### DatetimeIndex 中的各个标量值是 pandas 的 Timestamp 对象:

In [23]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

#### 只要有需要，TimeStamp 可以随时自动转换为 datetime 对象。此外，它还可以 存储频率信息(如果有的话)，且知道如何执行时区转换以及其他操作。稍后 将对此进行详细讲解。


### 索引、选取、子集构造

#### 当你根据标签索引选取数据时，时间序列和其它的 pandas.Series 很像:

In [24]:
stamp = ts.index[2]
ts[stamp]

0.3730298638351997

#### 还有一种更为方便的用法:传入一个可以被解释为日期的字符串:

In [25]:
ts['1/10/2011']

-0.3366885026963404

#### 对于较长的时间序列，只需传入“年”或“年月”即可轻松选取数据的切片:

In [26]:
longer_ts = pd.Series(np.random.randn(1000),index = pd.date_range('1/1/2000',periods = 1000))
longer_ts

2000-01-01    2.183170
2000-01-02    1.201187
2000-01-03    0.693444
2000-01-04   -1.718743
2000-01-05    0.728987
2000-01-06    1.236979
2000-01-07    0.226939
2000-01-08   -0.633026
2000-01-09    1.676673
2000-01-10   -0.433811
2000-01-11   -0.869071
2000-01-12    0.894152
2000-01-13    0.489174
2000-01-14    0.173658
2000-01-15    0.846037
2000-01-16    0.197252
2000-01-17    1.477008
2000-01-18   -1.556448
2000-01-19    1.738487
2000-01-20   -2.230658
2000-01-21    0.796059
2000-01-22    2.552825
2000-01-23   -1.521627
2000-01-24    0.792009
2000-01-25    1.380994
2000-01-26    0.241026
2000-01-27    0.869646
2000-01-28    0.848687
2000-01-29    1.027567
2000-01-30    0.092433
                ...   
2002-08-28    1.202865
2002-08-29   -0.980272
2002-08-30   -0.409982
2002-08-31    0.866817
2002-09-01    0.200926
2002-09-02    0.627184
2002-09-03   -0.886543
2002-09-04   -0.111983
2002-09-05    1.018946
2002-09-06   -0.619766
2002-09-07    1.067136
2002-09-08    0.770580
2002-09-09 

In [27]:
longer_ts['2001']

2001-01-01    0.609747
2001-01-02   -1.150369
2001-01-03    0.200562
2001-01-04    0.455702
2001-01-05    0.483372
2001-01-06   -0.534408
2001-01-07   -1.140054
2001-01-08   -1.010421
2001-01-09   -0.006864
2001-01-10   -0.022349
2001-01-11    0.953559
2001-01-12   -0.128604
2001-01-13    1.469755
2001-01-14    1.536848
2001-01-15   -0.056293
2001-01-16    0.919421
2001-01-17   -0.102166
2001-01-18    0.244071
2001-01-19   -1.385159
2001-01-20    0.467005
2001-01-21   -1.057327
2001-01-22    0.060749
2001-01-23   -0.714117
2001-01-24    0.002148
2001-01-25   -1.187596
2001-01-26    1.387348
2001-01-27   -0.881742
2001-01-28    2.686238
2001-01-29    0.607304
2001-01-30   -1.234905
                ...   
2001-12-02   -0.244722
2001-12-03    0.294545
2001-12-04    0.267344
2001-12-05   -1.724967
2001-12-06    2.880642
2001-12-07   -0.528650
2001-12-08   -0.386812
2001-12-09   -0.987154
2001-12-10    1.413348
2001-12-11   -0.300539
2001-12-12    0.164849
2001-12-13   -0.924797
2001-12-14 

#### 这里，字符串“2001”被解释成年，并根据它选取时间区间。指定月也同样奏 效:

In [28]:
longer_ts['2001-05']

2001-05-01    0.476222
2001-05-02    1.067093
2001-05-03   -0.993658
2001-05-04   -0.290496
2001-05-05    0.496013
2001-05-06   -0.596108
2001-05-07   -0.693683
2001-05-08   -1.220181
2001-05-09   -1.534602
2001-05-10    0.907214
2001-05-11   -1.159111
2001-05-12   -1.816547
2001-05-13    0.579047
2001-05-14   -0.375964
2001-05-15    0.734637
2001-05-16    0.315080
2001-05-17    2.217867
2001-05-18    0.203259
2001-05-19    0.958090
2001-05-20   -1.573871
2001-05-21   -0.845816
2001-05-22   -0.043164
2001-05-23   -0.393585
2001-05-24    1.630075
2001-05-25   -0.059596
2001-05-26    0.593113
2001-05-27   -1.371164
2001-05-28   -0.870213
2001-05-29    0.977082
2001-05-30    0.332667
2001-05-31   -0.614436
Freq: D, dtype: float64

#### datetime 对象也可以进行切片:

In [29]:
ts[datetime(2011, 1, 7):]

2011-01-07    0.373030
2011-01-08   -1.286663
2011-01-10   -0.336689
2011-01-12    0.109767
dtype: float64

#### 由于大部分时间序列数据都是按照时间先后排序的，因此你也可以用不存在于该时间序列中的时间戳对其进行切片(即范围查询):


In [30]:
ts


2011-01-02    1.674547
2011-01-05   -0.793054
2011-01-07    0.373030
2011-01-08   -1.286663
2011-01-10   -0.336689
2011-01-12    0.109767
dtype: float64

In [31]:
ts['1/6/2011':'1/11/2011']

2011-01-07    0.373030
2011-01-08   -1.286663
2011-01-10   -0.336689
dtype: float64

#### 跟之前一样，你可以传入字符串日期、datetime 或 Timestamp。注意，这样切 片所产生的是源时间序列的视图，跟 NumPy 数组的切片运算是一样的。这意味着，没有数据被复制，对切片进行修改会反映到原始数据上。 此外，还有一个等价的实例方法也可以截取两个日期之间 TimeSeries:


In [32]:
ts.truncate(after='1/9/2011')

2011-01-02    1.674547
2011-01-05   -0.793054
2011-01-07    0.373030
2011-01-08   -1.286663
dtype: float64

#### 面这些操作对 DataFrame 也有效。例如，对 DataFrame 的行进行索引:

In [34]:
dates = pd.date_range('1/1/2000',periods = 100,freq = 'W-WED')
long_df = pd.DataFrame(np.random.randn(100,4),index = dates,columns=['Colorado','Texas','New York','Ohio'])
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,1.087962,-0.217506,-0.159058,-0.687916
2001-05-09,0.278123,-1.931497,-2.431630,-0.500524
2001-05-16,-0.787364,-0.847830,-0.168658,-0.339712
2001-05-23,0.194604,-0.546186,-0.740795,-0.765588
2001-05-30,-0.063035,-1.516072,0.217782,-1.177296


### 带有重复索引的时间序列

#### 在某些应用场景中，可能会存在多个观测数据落在同一个时间点上的情况。下面就是一个例子:


In [35]:
dates = pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000','1/3/2000'])
dup_ts = pd.Series(np.arange(5),index = dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

#### 通过检查索引的 is_unique 属性，我们就可以知道它是不是唯一的:

In [36]:
dup_ts.index.is_unique

False

#### 对这个时间序列进行索引，要么产生标量值，要么产生切片，具体要看所选的时间点是否重复:


In [37]:
dup_ts['1/3/2000']

4

In [38]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

#### 假设你想要对具有非唯一时间戳的数据进行聚合。一个办法是使用 groupby， 并传入 level=0:


In [39]:
ground = dup_ts.groupby(level=0)
ground.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [40]:
ground.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

### 11.3 日期的范围、频率以及移动

#### pandas 中的原生时间序列一般被认为是不规则的，也就是说，它们没有固定的 频率。对于大部分应用程序而言，这是无所谓的。但是，它常常需要以某种相对固定的频率进行分析，比如每日、每月、每 15 分钟等(这样自然会在时间序 列中引入缺失值)。幸运的是，pandas 有一整套标准时间序列频率以及用于重 采样、频率推断、生成固定频率日期范围的工具。例如，我们可以将之前那个 时间序列转换为一个具有固定频率(每日)的时间序列，只需调用 resample 即 可:

In [41]:
ts

2011-01-02    1.674547
2011-01-05   -0.793054
2011-01-07    0.373030
2011-01-08   -1.286663
2011-01-10   -0.336689
2011-01-12    0.109767
dtype: float64

In [42]:
resampler = ts.resample('D')

#### 字符串“D”是每天的意思。频率的转换(或重采样)是一个比较大的主题，稍后将专门用一节来进行讨论 (11.6 小节)。这里，我将告诉你如何使用基本的频率和它的倍数。


#### 生成日期范围

#### 虽然我之前用的时候没有明说，但你可能已经猜到 pandas.date_range 可用于 根据指定的频率生成指定长度的 DatetimeIndex:


In [43]:
index = pd.date_range('2012-04-01','2012-06-01')
index 

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

#### 默认情况下，date_range 会产生按天计算的时间点。如果只传入起始或结束日 期，那就还得传入一个表示一段时间的数字:


In [44]:
pd.date_range(start ='2012-04-01',periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [45]:
pd.date_range(end='2012-06-01',periods = 20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

#### 起始和结束日期定义了日期索引的严格边界。例如，如果你想要生成一个由每 月最后一个工作日组成的日期索引，可以传入"BM"频率(表示 business end of month，表 11-4 是频率列表)，这样就只会包含时间间隔内(或刚好在边界 上的)符合频率要求的日期:


In [46]:
pd.date_range('2000-01-01','2000-12-01',freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

#### date_range 默认会保留起始和结束时间戳的时间信息(如果有的话):

In [47]:
pd.date_range('2012-05-02 12:56:31',periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

#### 有时，虽然起始和结束日期带有时间信息，但你希望产生一组被规范化 (normalize)到午夜的时间戳。normalize 选项即可实现该功能:


In [48]:
pd.date_range('2012-05-02 12:56:31',periods = 5,normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 频率和日期偏移量

#### pandas 中的频率是由一个基础频率(base frequency)和一个乘数组成的。基 础频率通常以一个字符串别名表示，比如"M"表示每月，"H"表示每小时。对于 每个基础频率，都有一个被称为日期偏移量(date offset)的对象与之对应。 例如，按小时计算的频率可以用 Hour 类表示:


In [49]:
from pandas.tseries.offsets import Hour,Minute
hour = Hour()
hour

<Hour>

#### 传入一个整数即可定义偏移量的倍数:

In [50]:
four_hours = Hour(4)
four_hours

<4 * Hours>

#### 一般来说，无需明确创建这样的对象，只需使用诸如"H"或"4H"这样的字符串别 名即可。在基础频率前面放上一个整数即可创建倍数:

In [51]:
pd.date_range('2000-01-01','2000-01-03 23:59',freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

#### 大部分偏移量对象都可通过加法进行连接:

In [52]:
Hour(2)+Minute(30)

<150 * Minutes>

#### 同理，你也可以传入频率字符串(如"2h30min")，这种字符串可以被高效地解 析为等效的表达式:
 

In [53]:
pd.date_range('2000-01-01',periods=10,freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

#### 有些频率所描述的时间点并不是均匀分隔的。例如，"M"(日历月末)和"BM" (每月最后一个工作日)就取决于每月的天数，对于后者，还要考虑月末是不 是周末。由于没有更好的术语，我将这些称为锚点偏移量(anchored offset)。


#### 笔记:用户可以根据实际需求自定义一些频率类以便提供 pandas 所没有的日期 逻辑，但具体的细节超出了本书的范围。


### WOM 日期

#### WOM(Week Of Month)是一种非常实用的频率类，它以 WOM 开头。它使你能获 得诸如“每月第 3 个星期五”之类的日期:


In [54]:
rng = pd.date_range('2012-01-01','2012-09-01',freq = 'WOM-3FRI')
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### 移动(超前和滞后)数据

#### 移动(shifting)指的是沿着时间轴将数据前移或后移。Series 和 DataFrame 都有一个 shift 方法用于执行单纯的前移或后移操作，保持索引不变:


In [55]:
ts = pd.Series(np.random.randn(4),index = pd.date_range('1/1/2000',periods=4,freq='M'))
ts

2000-01-31   -0.467139
2000-02-29    0.899186
2000-03-31   -0.573816
2000-04-30    0.628174
Freq: M, dtype: float64

In [56]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.467139
2000-04-30    0.899186
Freq: M, dtype: float64

In [57]:
ts.shift(-2)

2000-01-31   -0.573816
2000-02-29    0.628174
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

#### 当我们这样进行移动时，就会在时间序列的前面或后面产生缺失数据。

#### shift 通常用于计算一个时间序列或多个时间序列(如 DataFrame 的列)中的 百分比变化。可以这样表达:


In [58]:
ts/ts.shift(1)-1

2000-01-31         NaN
2000-02-29   -2.924879
2000-03-31   -1.638151
2000-04-30   -2.094730
Freq: M, dtype: float64

#### 由于单纯的移位操作不会修改索引，所以部分数据会被丢弃。因此，如果频率 已知，则可以将其传给 shift 以便实现对时间戳进行位移而不是对数据进行简 单位移:


In [59]:
ts.shift(2,freq='M')

2000-03-31   -0.467139
2000-04-30    0.899186
2000-05-31   -0.573816
2000-06-30    0.628174
Freq: M, dtype: float64

#### 这里还可以使用其他频率，于是你就能非常灵活地对数据进行超前和滞后处理了:


In [60]:
ts.shift(3,freq='D')

2000-02-03   -0.467139
2000-03-03    0.899186
2000-04-03   -0.573816
2000-05-03    0.628174
dtype: float64

In [61]:
ts.shift(1,freq='90T')

2000-01-31 01:30:00   -0.467139
2000-02-29 01:30:00    0.899186
2000-03-31 01:30:00   -0.573816
2000-04-30 01:30:00    0.628174
Freq: M, dtype: float64

### 通过偏移量对日期进行位移

#### pandas 的日期偏移量还可以用在 datetime 或 Timestamp 对象上:

In [63]:
from pandas.tseries.offsets import Day,MonthEnd
now = datetime(2011,11,17)
now + 3*Day()

Timestamp('2011-11-20 00:00:00')

#### 如果加的是锚点偏移量(比如 MonthEnd)，第一次增量会将原日期向前滚动到 符合频率规则的下一个日期:


In [64]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

#### 通过锚点偏移量的 rollforward 和 rollback 方法，可明确地将日期向前或向后 “滚动”:


In [65]:
offset = MonthEnd()
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [66]:
offset.rollback(now)


Timestamp('2011-10-31 00:00:00')

#### 日期偏移量还有一个巧妙的用法，即结合 groupby 使用这两个“滚动”方法:

In [67]:
ts = pd.Series(np.random.randn(20),index = pd.date_range('1/15/2000',periods=20,freq='4d'))
ts

2000-01-15    0.999693
2000-01-19   -0.406268
2000-01-23   -1.935245
2000-01-27   -0.260646
2000-01-31   -0.861704
2000-02-04   -1.396065
2000-02-08    0.008931
2000-02-12    0.210462
2000-02-16    0.507247
2000-02-20   -1.138882
2000-02-24   -1.173001
2000-02-28   -0.327023
2000-03-03   -0.889626
2000-03-07   -0.835115
2000-03-11    2.116177
2000-03-15   -1.887910
2000-03-19   -0.507900
2000-03-23   -2.227769
2000-03-27   -2.630440
2000-03-31   -0.675545
Freq: 4D, dtype: float64

In [69]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.492834
2000-02-29   -0.472619
2000-03-31   -0.942266
dtype: float64

#### 当然，更简单、更快速地实现该功能的办法是使用 resample(11.6 小节将对此 进行详细介绍):


In [70]:
ts.resample('M').mean()

2000-01-31   -0.492834
2000-02-29   -0.472619
2000-03-31   -0.942266
Freq: M, dtype: float64

#### 11.4 时区处理

#### 时间序列处理工作中最让人不爽的就是对时区的处理。许多人都选择以协调世 界时(UTC，它是格林尼治标准时间(Greenwich Mean Time)的接替者，目前 已经是国际标准了)来处理时间序列。时区是以 UTC 偏移量的形式表示的。例 如，夏令时期间，纽约比 UTC 慢 4 小时，而在全年其他时间则比 UTC 慢 5 小 时。

#### 在 Python 中，时区信息来自第三方库 pytz，它使 Python 可以使用 Olson 数据 库(汇编了世界时区信息)。这对历史数据非常重要，这是因为由于各地政府 的各种突发奇想，夏令时转变日期(甚至 UTC 偏移量)已经发生过多次改变 了。就拿美国来说，DST 转变时间自 1900 年以来就改变过多次!有关 pytz 库的更多信息，请查阅其文档。就本书而言，由于 pandas 包装了 pytz 的功能，因此你可以不用记忆其 API，只要记得时区的名称即可。时区名 可以在 shell 中看到，也可以通过文档查看:


In [71]:
import pytz

In [72]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

#### 要从 pytz 中获取时区对象，使用 pytz.timezone 即可:

In [73]:
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

#### pandas 中的方法既可以接受时区名也可以接受这些对象。

### 时区本地化和转换

#### 默认情况下，pandas 中的时间序列是单纯的(naive)时区。看看下面这个时 间序列:

In [74]:
rng = pd.date_range('3/9/2012 9:30',periods=6,freq='D')
ts = pd.Series(np.random.randn(len(rng)),index = rng)
ts

2012-03-09 09:30:00   -1.493392
2012-03-10 09:30:00    0.533011
2012-03-11 09:30:00   -1.170657
2012-03-12 09:30:00   -2.074994
2012-03-13 09:30:00    0.231734
2012-03-14 09:30:00   -1.264399
Freq: D, dtype: float64

#### 其索引的 tz 字段为 None:

In [75]:
print(ts.index.tz)

None


#### 可以用时区集生成日期范围:

In [76]:
pd.date_range('3/9/2012 9:30',periods = 10,freq='D',tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

#### 从单纯到本地化的转换是通过 tz_localize 方法处理的:

In [77]:
ts

2012-03-09 09:30:00   -1.493392
2012-03-10 09:30:00    0.533011
2012-03-11 09:30:00   -1.170657
2012-03-12 09:30:00   -2.074994
2012-03-13 09:30:00    0.231734
2012-03-14 09:30:00   -1.264399
Freq: D, dtype: float64

In [78]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00   -1.493392
2012-03-10 09:30:00+00:00    0.533011
2012-03-11 09:30:00+00:00   -1.170657
2012-03-12 09:30:00+00:00   -2.074994
2012-03-13 09:30:00+00:00    0.231734
2012-03-14 09:30:00+00:00   -1.264399
Freq: D, dtype: float64

In [79]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

#### 一旦时间序列被本地化到某个特定时区，就可以用 tz_convert 将其转换到别的 时区了:


In [80]:
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00   -1.493392
2012-03-10 04:30:00-05:00    0.533011
2012-03-11 05:30:00-04:00   -1.170657
2012-03-12 05:30:00-04:00   -2.074994
2012-03-13 05:30:00-04:00    0.231734
2012-03-14 05:30:00-04:00   -1.264399
Freq: D, dtype: float64

#### 对于上面这种时间序列(它跨越了美国东部时区的夏令时转变期)，我们可以 将其本地化到 EST，然后转换为 UTC 或柏林时间:


In [81]:
ts_eastern = ts.tz_localize('America/New_York')
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00   -1.493392
2012-03-10 14:30:00+00:00    0.533011
2012-03-11 13:30:00+00:00   -1.170657
2012-03-12 13:30:00+00:00   -2.074994
2012-03-13 13:30:00+00:00    0.231734
2012-03-14 13:30:00+00:00   -1.264399
Freq: D, dtype: float64

In [82]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00   -1.493392
2012-03-10 15:30:00+01:00    0.533011
2012-03-11 14:30:00+01:00   -1.170657
2012-03-12 14:30:00+01:00   -2.074994
2012-03-13 14:30:00+01:00    0.231734
2012-03-14 14:30:00+01:00   -1.264399
Freq: D, dtype: float64

#### 操作时区意识型 Timestamp 对象

#### 跟时间序列和日期范围差不多，独立的 Timestamp 对象也能被从单纯型 (naive)本地化为时区意识型(time zone-aware)，并从一个时区转换到另 一个时区:


In [83]:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('America/New_York')


Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

#### 在创建 Timestamp 时，还可以传入一个时区信息:

In [84]:
stamp_moscow = pd.Timestamp('2011-03-12 04:00',tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

#### 时区意识型 Timestamp 对象在内部保存了一个 UTC 时间戳值(自 UNIX 纪元 (1970 年 1 月 1 日)算起的纳秒数)。这个 UTC 值在时区转换过程中是不会发 生变化的:
 

In [85]:
stamp_utc.value

1299902400000000000

In [86]:
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

#### 当使用 pandas 的 DateOffset 对象执行时间算术运算时，运算过程会自动关注 是否存在夏令时转变期。这里，我们创建了在 DST 转变之前的时间戳。首先， 来看夏令时转变前的 30 分钟:


In [87]:
from pandas.tseries.offsets import Hour
stamp = pd.Timestamp('2012-03-12 01:30',tz = 'US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [88]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

#### 然后，夏令时转变前 90 分钟:

In [89]:
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [90]:
stamp + 2*Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

### 不同时区之间的运算


#### 如果两个时间序列的时区不同，在将它们合并到一起时，最终结果就会是 UTC。由于时间戳其实是以 UTC 存储的，所以这是一个很简单的运算，并不需要 发生任何转换:


In [91]:
rng = pd.date_range('3/7/2012 9:30',periods=10,freq='B')
ts = pd.Series(np.random.randn(len(rng)),index = rng)
ts

2012-03-07 09:30:00   -1.427777
2012-03-08 09:30:00    0.427203
2012-03-09 09:30:00   -1.789985
2012-03-12 09:30:00    2.202751
2012-03-13 09:30:00    0.276093
2012-03-14 09:30:00   -0.083592
2012-03-15 09:30:00   -1.270627
2012-03-16 09:30:00    1.750692
2012-03-19 09:30:00    0.098877
2012-03-20 09:30:00    0.177196
Freq: B, dtype: float64

In [94]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 + ts2
result.index 

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

### 11.5 时期及其算术运算

#### 时期(period)表示的是时间区间，比如数日、数月、数季、数年等。Period 类所表示的就是这种数据类型，其构造函数需要用到一个字符串或整数，以及 表 11-4 中的频率:

In [95]:
p = pd.Period(2007,freq='A-DEC')
p

Period('2007', 'A-DEC')

#### 这里，这个Period对象表示的是从2007年1月1日到2007年12月31日之间 的整段时间。只需对 Period 对象加上或减去一个整数即可达到根据其频率进行 位移的效果:

In [96]:
p + 5

Period('2012', 'A-DEC')

In [97]:
p-2

Period('2005', 'A-DEC')

#### 如果两个 Period 对象拥有相同的频率，则它们的差就是它们之间的单位数量:

In [98]:
pd.Period(2014,freq= 'A-DEC') - p

<7 * YearEnds: month=12>

#### period_range 函数可用于创建规则的时期范围:

In [99]:
rng = pd.period_range('2000-01-01','2000-06-30',freq = 'M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

#### PeriodIndex 类保存了一组 Period，它可以在任何 pandas 数据结构中被用作轴 索引:


In [100]:
pd.Series(np.random.randn(6),index = rng)

2000-01    0.785062
2000-02    0.443521
2000-03    0.675207
2000-04   -1.299149
2000-05    0.003778
2000-06    0.557434
Freq: M, dtype: float64

#### 如果你有一个字符串数组，你也可以使用 PeriodIndex 类:

In [101]:
values = ['2001Q3','2002Q2','2003Q1']
index = pd.PeriodIndex(values,freq='Q-DEC')
index 

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

#### Period 和 PeriodIndex 对象都可以通过其 asfreq 方法被转换成别的频率。假 设我们有一个年度时期，希望将其转换为当年年初或年末的一个月度时期。该 任务非常简单:


In [102]:
p = pd.Period('2007',freq='A-DEC')
p

Period('2007', 'A-DEC')

In [104]:
p.asfreq('M',how='start')


Period('2007-01', 'M')

#### 你可以将 Period('2007','A-DEC')看做一个被划分为多个月度时期的时间段中 的游标。图 11-1 对此进行了说明。对于一个不以 12 月结束的财政年度，月度 子时期的归属情况就不一样了:


In [105]:
p = pd.Period('2007',freq = 'A-JUN')
p

Period('2007', 'A-JUN')

In [106]:
p.asfreq('M','start')

Period('2006-07', 'M')

####  在将高频率转换为低频率时，超时期(superperiod)是由子时期 (subperiod)所属的位置决定的。例如，在 A-JUN 频率中，月份“2007 年 8 月”实际上是属于周期“2008 年”的:


In [107]:
p = pd.Period('Aug-2007','M')
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

#### 完整的 PeriodIndex 或 TimeSeries 的频率转换方式也是如此:

In [109]:
rng = pd.period_range('2006','2009',freq = 'A-DEC')
ts = pd.Series(np.random.randn(len(rng)),index = rng)
ts

2006   -0.560401
2007    1.099945
2008   -0.755806
2009   -0.520459
Freq: A-DEC, dtype: float64

In [110]:
ts.asfreq('M', how='start')

2006-01   -0.560401
2007-01    1.099945
2008-01   -0.755806
2009-01   -0.520459
Freq: M, dtype: float64

#### 这里，根据年度时期的第一个月，每年的时期被取代为每月的时期。如果我们 想要每年的最后一个工作日，我们可以使用“B”频率，并指明想要该时期的末 尾:


In [111]:
ts.asfreq('B',how='end')

2006-12-29   -0.560401
2007-12-31    1.099945
2008-12-31   -0.755806
2009-12-31   -0.520459
Freq: B, dtype: float64

#### 按季度计算的时期频率

#### 季度型数据在会计、金融等领域中很常见。许多季度型数据都会涉及“财年 末”的概念，通常是一年 12 个月中某月的最后一个日历日或工作日。就这一点 来说，时期"2012Q4"根据财年末的不同会有不同的含义。pandas 支持 12 种可 能的季度型频率，即 Q-JAN 到 Q-DEC:


In [112]:
p = pd.Period('2012Q4',freq='Q-JAN')
p

Period('2012Q4', 'Q-JAN')

#### 在以 1 月结束的财年中，2012Q4 是从 11 月到 1 月(将其转换为日型频率就明 白了)。图 11-2 对此进行了说明:



In [113]:
p.asfreq('D','start')

Period('2011-11-01', 'D')

In [114]:
p.asfreq('D','end')

Period('2012-01-31', 'D')

#### 因此，Period 之间的算术运算会非常简单。例如，要获取该季度倒数第二个工作日下午 4 点的时间戳，你可以这样:

In [116]:
p4pm = (p.asfreq('B','e')-1).asfreq('T','s')+16*60
p4pm

Period('2012-01-30 16:00', 'T')

In [117]:
p4pm.to_timestamp()

Timestamp('2012-01-30 16:00:00')

#### period_range 可用于生成季度型范围。季度型范围的算术运算也跟上面是一样 的:


In [119]:
rng = pd.period_range('2011Q3','2012Q4',freq='Q-JAN')
ts = pd.Series(np.arange(len(rng)),index=rng)
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int64

In [122]:
new_rng = (rng.asfreq('B', 'e') - 1).asfreq('T', 's') + 16* 60
ts.index = new_rng.to_timestamp()
ts

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int64

### 将 Timestamp 转换为 Period(及其反向过程)

#### 通过使用 to_period 方法，可以将由时间戳索引的 Series 和 DataFrame 对象转 换为以时期索引:


In [124]:
rng = pd.date_range('2000-01-01',periods=3,freq='M')
ts = pd.Series(np.random.randn(3),index = rng)
ts

2000-01-31    0.101677
2000-02-29   -1.047653
2000-03-31    0.701950
Freq: M, dtype: float64

In [125]:
pts = ts.to_period()
pts

2000-01    0.101677
2000-02   -1.047653
2000-03    0.701950
Freq: M, dtype: float64

#### 由于时期指的是非重叠时间区间，因此对于给定的频率，一个时间戳只能属于 一个时期。新 PeriodIndex 的频率默认是从时间戳推断而来的，你也可以指定 任何别的频率。结果中允许存在重复时期:


In [126]:
rng = pd.date_range('1/29/2000',periods = 6,freq = 'D')
ts2 = pd.Series(np.random.randn(6),index = rng)
ts2

2000-01-29   -0.523170
2000-01-30   -0.392553
2000-01-31   -1.643643
2000-02-01   -0.022528
2000-02-02    0.047346
2000-02-03    0.813889
Freq: D, dtype: float64

In [127]:
ts2.to_period('M')

2000-01   -0.523170
2000-01   -0.392553
2000-01   -1.643643
2000-02   -0.022528
2000-02    0.047346
2000-02    0.813889
Freq: M, dtype: float64

#### 要转换回时间戳，使用 to_timestamp 即可:

In [128]:
pts

2000-01    0.101677
2000-02   -1.047653
2000-03    0.701950
Freq: M, dtype: float64

#### 通过数组创建 PeriodIndex

#### 固定频率的数据集通常会将时间信息分开存放在多个列中。例如，在下面这个宏观经济数据集中，年度和季度就分别存放在不同的列中:


### 11.6 重采样及频率转换

#### 重采样(resampling)指的是将时间序列从一个频率转换到另一个频率的处理 过程。将高频率数据聚合到低频率称为降采样(downsampling)，而将低频率 数据转换到高频率则称为升采样(upsampling)。并不是所有的重采样都能被 划分到这两个大类中。例如，将 W-WED(每周三)转换为 W-FRI 既不是降采样 也不是升采样。


#### pandas 对象都带有一个 resample 方法，它是各种频率转换工作的主力函数。 resample 有一个类似于 groupby 的 API，调用 resample 可以分组数据，然后会 调用一个聚合函数:

In [130]:
rng = pd.date_range('2000-01-01',periods=100,freq='D')
ts = pd.Series(np.random.randn(len(rng)),index = rng)
ts

2000-01-01   -2.144875
2000-01-02    1.618518
2000-01-03   -0.368348
2000-01-04   -0.323510
2000-01-05    0.705665
2000-01-06   -1.443802
2000-01-07    0.370770
2000-01-08    0.106879
2000-01-09   -0.097023
2000-01-10   -1.132052
2000-01-11   -0.055505
2000-01-12   -0.851985
2000-01-13    1.130984
2000-01-14   -2.165898
2000-01-15   -0.047495
2000-01-16   -1.164316
2000-01-17    1.231316
2000-01-18   -1.110849
2000-01-19   -0.138990
2000-01-20   -0.196846
2000-01-21    0.817326
2000-01-22   -0.567570
2000-01-23   -0.019005
2000-01-24    0.462190
2000-01-25   -0.613445
2000-01-26   -2.265682
2000-01-27    1.284460
2000-01-28   -0.709515
2000-01-29    0.398591
2000-01-30    0.054205
                ...   
2000-03-11   -1.476526
2000-03-12    0.324289
2000-03-13   -0.572273
2000-03-14   -0.574297
2000-03-15   -1.165440
2000-03-16   -0.807838
2000-03-17   -2.108470
2000-03-18    0.918027
2000-03-19    0.209381
2000-03-20    0.235800
2000-03-21   -1.014448
2000-03-22    1.317508
2000-03-23 

In [131]:
ts.resample('M').mean()

2000-01-31   -0.303474
2000-02-29    0.186931
2000-03-31   -0.161226
2000-04-30    0.276333
Freq: M, dtype: float64

In [132]:
ts.resample('M', kind='period').mean()

2000-01   -0.303474
2000-02    0.186931
2000-03   -0.161226
2000-04    0.276333
Freq: M, dtype: float64

#### resample 是一个灵活高效的方法，可用于处理非常大的时间序列。我将通过一 系列的示例说明其用法。表 11-5 总结它的一些选项。


##### freq                                 表示重采样频率的字符串或DeteOffset,例如‘M’
##### axis                                 重采样的轴 默认为axis=0
##### fill_method                    升采样如何插值，比如’fill‘或’bfill‘  默认不插值
##### closed                            在降采样中，各时间段的那一端是闭合（即包含）的，RIGHT或者LEFT，默认是RIGHT
##### label                               在降采样中如何设置聚合标签
##### kind                                聚合到期或时间戳，默认聚合道时间序列的索引类型

### 降采样

#### 将数据聚合到规律的低频率是一件非常普通的时间序列处理任务。待聚合的数 据不必拥有固定的频率，期望的频率会自动定义聚合的面元边界，这些面元用 于将时间序列拆分为多个片段。例如，要转换到月度频率('M'或'BM')，数据 需要被划分到多个单月时间段中。各时间段都是半开放的。一个数据点只能属 于一个时间段，所有时间段的并集必须能组成整个时间帧。在用 resample 对数 据进行降采样时，需要考虑两样东西:


In [ ]:
各区间哪边是闭合的。
如何标记各个聚合面元，用区间的开头还是末尾。

#### 为了说明，我们来看一些“1 分钟”数据:

In [134]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')
ts = pd.Series(np.arange(12),index = rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int64

#### 假设你想要通过求和的方式将这些数据聚合到“5 分钟”块中:

In [135]:
ts.resample('5min',closed = 'right').sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int64

#### 传入的频率将会以“5 分钟”的增量定义面元边界。默认情况下，面元的右边 界是包含的，因此 00:00 到 00:05 的区间中是包含 00:05 的。传入 closed='left'会让区间以左边界闭合:


#### OHLC 重采样

#### 金融领域中有一种无所不在的时间序列聚合方式，即计算各面元的四个值:第 一个值(open，开盘)、最后一个值(close，收盘)、最大值(high，最高) 以及最小值(low，最低)。传入 how='ohlc'即可得到一个含有这四种聚合值 的 DataFrame。整个过程很高效，只需一次扫描即可计算出结果:


In [136]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


### 升采样和插值

#### 在将数据从低频率转换到高频率时，就不需要聚合了。我们来看一个带有一些 周型数据的 DataFrame:


### 通过时期进行重采样

### 11.7 移动窗口函数

####  在移动窗口(可以带有指数衰减权数)上计算的各种统计函数也是一类常见于 时间序列的数组变换。这样可以圆滑噪音数据或断裂数据。我将它们称为移动 窗口函数(moving window function)，其中还包括那些窗口不定长的函数 (如指数加权移动平均)。跟其他统计函数一样，移动窗口函数也会自动排除 缺失值。

#### 开始之前，我们加载一些时间序列数据，将其重采样为工作日频率: